In [1]:
!pip install transformers
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00


In [4]:
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 model and tokenizer
model_name = "t5-small"  # You can also use "t5-base" or "t5-large" for larger versions
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# User question to be rephrased
user_question = "What is the capital of France?"

# Create the prompt in a text-to-text format
prompt = f"rephrase: {user_question}"

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate rephrased question
output = model.generate(**inputs)
rephrased_question = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the rephrased question
print("Original Question:", user_question)
print("Rephrased Question:", rephrased_question)


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Original Question: What is the capital of France?
Rephrased Question: rephrase: What is the capital of France?


In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import Dataset, DataLoader
import torch

class RephraseDataset(Dataset):
    def __init__(self, tokenizer, data_path, max_source_length=128, max_target_length=128):
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

        # Load and preprocess your dataset here
        df = pd.read_csv(data_path)
        self.input_texts = df["input_text"].tolist()
        self.target_texts = df["target_text"].tolist()

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, index):
        # Tokenize the input and target texts
        input_encoding = self.tokenizer(
            self.input_texts[index],
            max_length=self.max_source_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            self.target_texts[index],
            max_length=self.max_target_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "target_ids": target_encoding["input_ids"].flatten(),
        }


In [10]:
# Load the T5 model and tokenizer
model_name = "t5-base"  # You can use other variants like "t5-base" or "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


train_dataset = RephraseDataset(tokenizer, "rephrase_dataset.csv")
val_dataset = RephraseDataset(tokenizer, "rephrase_dataset.csv")




/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [17]:
from transformers import AdamW, get_linear_schedule_with_warmup

# Training configurations
batch_size = 16
learning_rate = 2e-5
num_epochs = 50
warmup_steps = 500


# Create DataLoader objects for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

loss_function = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Fine-tuning loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is : {device}")
model.to(device)

Device is : cuda


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [18]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_loader):
        inputs = batch["input_ids"].to(device)
        targets = batch["target_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate

        total_loss += loss.item()

        if step % 10 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, Step {step}/{len(train_loader)}, Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Avg Loss: {avg_loss:.4f}")

# Save the trained model
model.save_pretrained("t5_model_trained")
tokenizer.save_pretrained("t5_model_trained")

Epoch 1/50, Step 0/1, Loss: 0.0527
Epoch 1/50, Avg Loss: 0.0527
Epoch 2/50, Step 0/1, Loss: 0.0907
Epoch 2/50, Avg Loss: 0.0907
Epoch 3/50, Step 0/1, Loss: 0.0423
Epoch 3/50, Avg Loss: 0.0423
Epoch 4/50, Step 0/1, Loss: 0.0953
Epoch 4/50, Avg Loss: 0.0953
Epoch 5/50, Step 0/1, Loss: 0.0834
Epoch 5/50, Avg Loss: 0.0834
Epoch 6/50, Step 0/1, Loss: 0.0648
Epoch 6/50, Avg Loss: 0.0648
Epoch 7/50, Step 0/1, Loss: 0.0572
Epoch 7/50, Avg Loss: 0.0572
Epoch 8/50, Step 0/1, Loss: 0.1017
Epoch 8/50, Avg Loss: 0.1017
Epoch 9/50, Step 0/1, Loss: 0.0868
Epoch 9/50, Avg Loss: 0.0868
Epoch 10/50, Step 0/1, Loss: 0.0663
Epoch 10/50, Avg Loss: 0.0663
Epoch 11/50, Step 0/1, Loss: 0.0567
Epoch 11/50, Avg Loss: 0.0567
Epoch 12/50, Step 0/1, Loss: 0.0870
Epoch 12/50, Avg Loss: 0.0870
Epoch 13/50, Step 0/1, Loss: 0.0547
Epoch 13/50, Avg Loss: 0.0547
Epoch 14/50, Step 0/1, Loss: 0.0576
Epoch 14/50, Avg Loss: 0.0576
Epoch 15/50, Step 0/1, Loss: 0.0651
Epoch 15/50, Avg Loss: 0.0651
Epoch 16/50, Step 0/1, Loss:

('t5_model_trained/tokenizer_config.json',
 't5_model_trained/special_tokens_map.json',
 't5_model_trained/spiece.model',
 't5_model_trained/added_tokens.json')

In [22]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# # Load the trained T5 model and tokenizer
model_name = "t5_model_trained"
tokenizer_pre = T5Tokenizer.from_pretrained(model_name)
model_pre = T5ForConditionalGeneration.from_pretrained(model_name)

# New input text to be rephrased
input_text = "What is PAD "

# Tokenize the input text
input_encoding = tokenizer_pre(
    input_text,
    max_length=128,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Generate rephrased output
output = model_pre.generate(
    input_ids=input_encoding["input_ids"],
    attention_mask=input_encoding["attention_mask"],
    max_length=128,
    num_beams=4,  # You can adjust the number of beams for beam search
    early_stopping=True,
)

# Decode the generated output into human-readable text
rephrased_text = tokenizer_pre.decode(output[0], skip_special_tokens=True)

# Print the rephrased question
print("Input Text:", input_text)
print("Rephrased Text:", rephrased_text)


Input Text: What is PAD 
Rephrased Text: What is Personal Account Deleaning?


In [19]:
torch.cuda.empty_cache()